In [22]:
import pandas as pd
import re
import os
import pingouin as pg

In [29]:
# Set working directory
os.chdir('/Users/nsusser/Desktop/Github/happyDB/')

# Load the main data and reverse-coded items
results = pd.read_csv('data/filtered_ratings.csv')  

reverse_coded_items_df = pd.read_csv('Profiles/reversed_column_names.csv')

In [24]:
# Display the first few rows of both datasets
print(results.columns.to_list())


['hmid', 'cleaned_hm', 'PERMA_Accomplishment_the_speaker_felt_they_were_making_progress_towards_accomplishing_their_goals?', 'PERMA_Engagement_the_speaker_became_absorbed_in_what_they_were_doing?', 'PERMA_Positive_Emotion_the_speaker_felt_joyful?', 'PERMA_Negative_emotion_the_speaker_felt_anxious?', 'PERMA_Accomplishment_the_speaker_achieved_the_important_goals_they_set_for_themselves?', 'PERMA_Health_the_speaker_perceived_their_health_positively?', 'PERMA_Meaning_the_speaker_felt_their_life_was_purposeful_and_meaningful?', 'PERMA_Relationships_the_speaker_received_help_and_support_from_others_when_needed?', 'PERMA_Meaning_the_speaker_felt_what_they_did_in_life_was_valuable_and_worthwhile?', 'PERMA_Engagement_the_speaker_felt_excited_and_interested_in_things?', 'PERMA_Lonliness_the_speaker_felt_lonely_in_their_daily_life?', 'PERMA_Health_the_speaker_felt_satisfied_with_their_current_physical_health?', 'PERMA_Positive_Emotion_the_speaker_felt_positive?', 'PERMA_Negative_emotion_the_spea

In [30]:
reversed_columns = reverse_coded_items_df['flat_name'].tolist()

max_value = 7

for col in reversed_columns:
    if col in results.columns:
        results[col] = max_value + 1 - results[col]
        print(f"✔ Reversed: {col}")
    else:
        print(f"❌ Not found in results: {col}")




✔ Reversed: CIT_Autonomy_-_Control_other_people_decided_most_of_the_speaker's_life_decisions?
✔ Reversed: CIT_Autonomy_-_Control_the_life_choices_the_speaker_made_were_not_really_theirs?
✔ Reversed: CIT_Autonomy_-_Control_other_people_decided_what_the_speaker_could_and_could_not_do?
✔ Reversed: CIT_Subjective_Well-Being_-_Negative_Feelings_the_speaker_felt_negative_most_of_the_time?
✔ Reversed: CIT_Subjective_Well-Being_-_Negative_Feelings_the_speaker_experienced_unhappy_feelings_most_of_the_time?
✔ Reversed: CIT_Subjective_Well-Being_-_Negative_Feelings_the_speaker_felt_bad_most_of_the_time?
✔ Reversed: PWB_Autonomy_the_speaker_was_not_afraid_to_voice_their_opinions,_even_when_they_were_in_opposition_to_others?
✔ Reversed: PWB_Personal_Growth_the_speaker_was_continuously_in_the_process_of_learning,_changing,_and_growing?
✔ Reversed: PWB_Environmental_Mastery_the_speaker_felt_in_charge_of_the_situation?
✔ Reversed: PWB_Positive_Relations_the_speaker_would_be_described_as_a_giving_perso

In [19]:
# Step 2: Define specific groupings for analysis
groupings = {
    "Accomplishment and Mastery": [
        ("PERMA", "Accomplishment"),
        ("CIT", "Mastery - Accomplishment"),
        ("CIT", "Mastery - Learning"),
        ("CIT", "Mastery - Self-Efficacy"),
        ("CIT", "Mastery - Self-Worth"),
        ("CIT", "Mastery - Skills"),
        ("PWB", "Environmental Mastery"),
        ("WBP", "Competence"),
        ("WBP", "Accomplishment"),
    ],
    "Autonomy and Control": [
        ("PWB", "Autonomy"),
        ("WBP", "Autonomy"),
        ("CIT", "Autonomy - Control"),
    ],
    "Engagement and Flow": [
        ("PERMA", "Engagement"),
        ("WBP", "Engagement"),
        ("CIT", "Engagement"),
    ],
    "Positive Emotions": [
        ("PERMA", "Positive Emotion"),
        ("WBP", "Positive Emotions"),
        ("PANAS", "Positive"),
    ],
    "Negative Emotions": [
        ("PERMA", "Negative emotion"),
        ("PANAS", "Negative"),
        ("CIT", "Subjective Well-Being - Negative Feelings"),
    ],
    "Meaning and Purpose": [
        ("PERMA", "Meaning"),
        ("WBP", "Meaning"),
        ("PWB", "Purpose in Life"),
        ("CIT", "Meaning"),
    ],
    "Positive Relationships": [
        ("PERMA", "Relationships"),
        ("PWB", "Positive Relations"),
        ("WBP", "Positive Relationships"),
        ("CIT", "Relationship - Belonging"),
        ("CIT", "Relationship - Community"),
        ("CIT", "Relationship - Respect"),
        ("CIT", "Relationship - Support"),
        ("CIT", "Relationship - Trust"),
    ],
    "Life Satisfaction and Well-Being": [
        ("SWLS", "LS"),
        ("WHO-5", "Well-Being"),
        ("CIT", "Subjective Well-Being - Life Satisfaction"),
    ]
}

In [20]:
# Step 2: Clean and sanitize groupings
# Define a function to clean and sanitize Scale and Dimension
def clean_text(text):
    return text.strip().replace(" ", "_")

# Clean the groupings dictionary
sanitized_groupings = {}
for group_name, dimensions in groupings.items():
    sanitized_dimensions = [(clean_text(scale), clean_text(dimension)) for scale, dimension in dimensions]
    sanitized_groupings[group_name] = sanitized_dimensions

# Step 3: Compute correlation matrices for each grouping
for group_name, dimensions in sanitized_groupings.items():
    group_items = []

    for scale, dimension in dimensions:
        # Match all items that belong to this scale and dimension
        # Allow for columns that start with the Scale_Dimension prefix, ignoring Items
        items = [
            col for col in results.columns
            if col.startswith(f"{scale}_{dimension}_")  # Match Scale and Dimension only
        ]
        group_items.extend(items)

    if len(group_items) > 1:  # Only calculate if there are multiple items in the group
        group_data = results[group_items]

        # Compute the correlation matrix for this group
        corr_matrix = group_data.corr()

        # Save the correlation matrix for this group
        output_path = f'analysis_data/item_correlations/Inter-Scales Dimension/{group_name}_correlation_matrix.xlsx'
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        corr_matrix.to_excel(output_path)

        # Optionally, print the correlation matrix for review
        # print(f"{group_name} Correlation Matrix:")
        # print(corr_matrix)


ModuleNotFoundError: No module named 'openpyxl'